#### Define Imports

In [1]:
#%pip install openai

In [2]:
! pip install gradio
! pip install transformers
! pip install torch
import gradio as gr
from transformers import pipeline, AutoModelForQuestionAnswering, AutoTokenizer
import pandas as pd
import torch

ERROR: Could not find a version that satisfies the requirement torch (from versions: none)
ERROR: No matching distribution found for torch


In [1]:
import sys
sys.path.insert(0,'../')
from environment import env
from environment import gpt_model_functions

#### API Call

In [2]:
query = 'Provide the state that Bryce Canyon is in'
endpoint, parkcode, intent, output = gpt_model_functions.api_call(query)
output

'Bryce Canyon National Park is located in Utah'

In [6]:
query = 'Where is Yosimte located?'
endpoint, parkcode, intent, output = gpt_model_functions.api_call(query)
output

'Yosemite National Park is located at 9039 Village Drive in Yosemite, California 95389'

In [7]:
query = 'What alerts are there at Bryce Canyon National Park?'
endpoint, parkcode, intent, output = gpt_model_functions.api_call(query)
intent

'alerts'

In [9]:
len(output)

2

In [ ]:
if intent == 'alerts':
    output = f'There are {len(output)} active alerts for ' 

In [8]:
output

,id,url,title,parkCode,description,category,relatedRoadEvents,lastIndexedDate
0,9840308A-02FB-4A2F-90A3-E5597EB3881B,https://www.nps.gov/brca/planyourvisit/conditi...,Main Road Status,brca,The main park road is fully open to Rainbow Po...,Information,[],2024-04-02 13:59:18.0
1,8833A24A-3F71-4E6E-9254-FF7743304115,https://www.nps.gov/brca/planyourvisit/basicin...,Bryce Canyon is Open - No Reservations Require...,brca,No reservations are required to enter Bryce Ca...,Information,[],2024-02-16 15:58:11.0


In [6]:
statecode_mapping = gpt_model_functions.statecode_mapping

In [11]:
f"{output['line1'][0]} {output['line2'][0]} {output['city'][0]}, {statecode_mapping(output['state'][0])} {output['postalCode'][0]}"
output = f"{output['fullName'][0]} is located at {output['line1'][0]} " + (f"{output['line2'][0]} " if output['line2'][0] else "") + f"in {output['city'][0]}, {statecode_mapping(output['state'][0])} {output['postalCode'][0]}"
output
#output = f"{output['fullName'][0]} is located at f"{output['line1'][0]} " + (f"{output['line2'][0]} " if output['line2'][0] else "") + f"{output['city'][0]}, {statecode_mapping(output['state'][0])} {output['postalCode'][0]}"


'Bryce Canyon National Park is located at Highway 63 Bryce Canyon National Park Bryce, Utah 84764'

In [9]:
query = 'Provide the description of Bryce Canyon National Park'
endpoint, parkcode, intent, output = gpt_model_functions.api_call(query)
intent

'description'

In [8]:
query = "What's the full name of Bryce Canyon"
endpoint, parkcode, intent, output = gpt_model_functions.api_call(query)
intent

'fullname'

In [4]:
query = 'How much are the enterance fees for Bryce Canyon National Park?'
endpoint, parkcode, intent, output = gpt_model_functions.api_call(query)
intent

'feespass'

In [5]:
query = 'What are the amenities at Bryce Canyon National Park?'
endpoint, parkcode, intent, output = gpt_model_functions.api_call(query)
intent

'amenities'

In [6]:
query = 'What events are happening at Bryce Canyon National Park?'
endpoint, parkcode, intent, output = gpt_model_functions.api_call(query)
intent

'events'

In [ ]:
model_name = "distilbert-base-uncased-distilled-squad"
model = AutoModelForQuestionAnswering.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Create a pipeline
nlp = pipeline("question-answering", model=model, tokenizer=tokenizer)

# Load the data
df = pd.read_csv('parks_data.csv')

# Initialize the model and tokenizer


# Function to generate answers
def answer_question(question, context):
    result = nlp(question=question, context=context)
    return result['answer']

# Function to get park context
def get_park_context(park_name):
    park = df[df['name'].str.contains(park_name, case=False, na=False)]
    if not park.empty:
        context = f"{park.iloc[0]['name']} is located at {park.iloc[0]['address']} in {park.iloc[0]['state']}. {park.iloc[0]['description']}"
        return context
    else:
        return "Sorry, I couldn't find the park you're looking for."

# Function to handle user queries
def handle_query(query):
    # Extract the entity (park name) from the query
    entity = extract_entity(query)
    if entity:
        context = get_park_context(entity)
        answer = answer_question(query, context)
        return answer
    else:
        return "Please specify a park in your question."

# Function to extract the entity (park name) from the query
def extract_entity(query):
    for park_name in df['name']:
        if park_name.lower() in query.lower():
            return park_name
    return None

# Example usage
query = "Tell me about Yellowstone National Park"
response = handle_query(query)
print(response)